In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential #model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D 

In [2]:
import pickle

#load data
with open("X.pickle", "rb") as pickle_in:
    X = pickle.load(pickle_in)
with open("y.pickle", "rb") as pickle_in:
    y = pickle.load(pickle_in)

# print(X.shape) # (84000, 200, 200, 1)

In [3]:
import numpy as np
from numpy.lib.format import open_memmap
#create memmap
Xm = np.memmap('X.memmap', dtype='float64', mode='w+', shape=X.shape)

# put data into memmap, normalizing it in the process
Xm[:] = X[:]/255.0

# load memmap

#Xm = np.memmap('X.memmap', dtype='float64', mode='r+', shape=(84000, 200, 200, 1))

In [4]:
model = Sequential() #create model
model.add(Conv2D(32, (3,3), activation='relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))

# do it again                                 (2nd layer)
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#                                             (3rd layer)
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#final dense layer                            
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Activation("relu"))

model.add(Dense(28))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 64)       0

In [5]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [7]:

model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy']
             )

model.fit(Xm[:30000], y[:30000], epochs=2, validation_split=0.2)   #limit batch size, scale with size of data

Epoch 1/2
750/750 [==============================] - 1701s 2s/step - loss: 1.4054 - accuracy: 0.5809 - val_loss: 0.5467 - val_accuracy: 0.8308
Epoch 2/2
750/750 [==============================] - 1301s 2s/step - loss: 0.3036 - accuracy: 0.9018 - val_loss: 0.2931 - val_accuracy: 0.9125


In [8]:
#calculate validation loss and accuracy
val_loss, val_acc = model.evaluate(Xm[:1000], y[:1000])
print(f"{val_loss=}, {val_acc=}")

32/32 [==============================] - 20s 460ms/step - loss: 0.0894 - accuracy: 0.9770
val_loss=0.08936435729265213, val_acc=0.9769999980926514


In [9]:
model.save("ASL01.model")

INFO:tensorflow:Assets written to: ASL01.model\assets


In [18]:
predictions = model.predict([Xm[45000:50000]])

In [22]:
# a case where the model is wrong, note that the second most likely possibility is correct (second greatest)
print(predictions[-1])
print(np.argmax(predictions[-1]))
print(y[49999])

[-13.846351   -10.725306    -5.955967    -4.5497355   -8.087611
  -6.0316033    6.4827504    6.6336465   -6.2914944    3.2519815
  -3.3955433   -8.320686    -6.290513    -3.1319869  -12.828208
   1.0138714   -6.5197973    0.91730154  -3.2615216  -10.731537
  -5.403344    -2.510316    -8.469812    -2.897812    -5.8702593
  -9.225175    -2.2867138  -12.792107  ]
7
6


In [ ]:
loaded = tf.keras.model.load("ASL01.model")